# Experiment 1: Augmentation

In [56]:
import os
import sys
import pickle
import glob
import time
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score

# Data preprocessing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
torch.manual_seed(0)
print(f'PyTorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

PyTorch version: 1.7.1
This notebook use cuda:0


In [6]:
# 파일 경로 사용자 정의
class path:
    data = '/opt/ml/input/original_data'
    train = f'{data}/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = f'{data}/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [7]:
BATCH_SIZE = 16
NUM_WORKERS = 2
LEARNING_RATE = 1e-4
EPOCHS = 3

## 1. Dataset Construction

In [8]:
class MaskDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def set_transform(self, transform):
        self.transform = transform
        
    def __getitem__(self, idx):
        current_data = self.data.iloc[idx]
        target = current_data.target
        image = Image.open(current_data.path)
        
        if self.transform:
            image = self.transform(image)
            
        return image, target
    def __len__(self):
        return len(self.data)

## 2. Augmentation Methods

- RandomHorizontalFlip
- CenterCrop
- RandomCrop
- Resize
- Normalize: ToTensor() 이후에 사용
- Color Jitter - Brightness
- Color Jitter - Contrast
- Color Jitter - Saturation
- To Tensor
- GrayScale
- FiveCrop, TenCrop
- GaussianBlur

In [9]:
# transforms 의 종류
dir(transforms)[:-13]

['CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor']

### 2.0. No Augmentaton

In [79]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [80]:
valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [81]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms)
valid_dataset.dataset.set_transform(valid_transforms)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=NUM_WORKERS, 
                          shuffle=True)
valid_loader = DataLoader(valid_dataset,
                          shuffle=False)                          

### 2.1. Augmentation 1

- CenterCrop (224, 224)
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [82]:
train_transforms1 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [83]:
valid_transforms1 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [85]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms1)
valid_dataset.dataset.set_transform(valid_transforms1)

# 데이터로더 생성
train_loader1 = DataLoader(train_dataset,
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader1 = DataLoader(valid_dataset,
                           shuffle=False)                          

### 2.2. Augmentation 2

- CenterCrop (384, 384)
- Resize (224, 224)
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [86]:
train_transforms2 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [87]:
valid_transforms2 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [88]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms2)
valid_dataset.dataset.set_transform(valid_transforms2)

# 데이터로더 생성
train_loader2 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader2 = DataLoader(valid_dataset,
                           shuffle=False)                          

### 2.3. Augmentation 3

- CenterCrop (384, 384)
- Resize(224, 224)
- GrayScale
- RandomHorizontalFlip
- RandomRotation
- Color Jitter
- ToTensor
- Normalize

In [89]:
train_transforms3 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation([-5, +5]),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.24]),
])

In [90]:
valid_transforms3 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.24]),
])

In [91]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms3)
valid_dataset.dataset.set_transform(valid_transforms3)

# 데이터로더 생성
train_loader3 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader3 = DataLoader(valid_dataset,
                           shuffle=False)                          

### 2.4. Augmentation 4

- CenterCrop (384, 384)
- Resize(224, 224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize

In [92]:
train_transforms4 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [93]:
valid_transforms4 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [94]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms4)
valid_dataset.dataset.set_transform(valid_transforms4)

# 데이터로더 생성
train_loader4 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader4 = DataLoader(valid_dataset,
                           shuffle=False)                          

### 2.5. Augmentation 5

- RandomResizedCenterCrop (224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize

In [95]:
train_transforms5 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [96]:
valid_transforms5 = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [97]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms5)
valid_dataset.dataset.set_transform(valid_transforms5)

# 데이터로더 생성
train_loader5 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader5 = DataLoader(valid_dataset,
                           shuffle=False)                

### 2.6. Augmentation 6

- CenterCrop (384, 384)
- Resize(224, 224)
- RandomHorizontalFlip
- Color Jitter
- ToTensor
- Normalize
- AddGuanssianNoise

In [98]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [99]:
train_transforms6 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.),
])

In [100]:
valid_transforms6 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [101]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms6)
valid_dataset.dataset.set_transform(valid_transforms6)

# 데이터로더 생성
train_loader6 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader6 = DataLoader(valid_dataset,
                           shuffle=False)                          

### 2.7. Augmentation 7

- CenterCrop (384, 384)
- Resize(224, 224)
- Color Jitter
- ToTensor
- Normalize
- AddGaussianNoise

In [102]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [103]:
train_transforms7 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.),
])

In [104]:
valid_transforms7 = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [105]:
# 데이터셋 생성
dataset = MaskDataset(f'{path.train}/train_modified.csv')

# 학습/검정 데이터셋으로 분리
n_train = int(len(dataset) * 0.8)
n_valid = len(dataset) - n_train
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

# Augmentation 설정
train_dataset.dataset.set_transform(train_transforms7)
valid_dataset.dataset.set_transform(valid_transforms7)

# 데이터로더 생성
train_loader7 = DataLoader(train_dataset, 
                           batch_size=BATCH_SIZE, 
                           num_workers=NUM_WORKERS, 
                           shuffle=True)
valid_loader7 = DataLoader(valid_dataset,
                           shuffle=False)                          

## 3. Compare Performance

In [106]:
def test_eval(model, valid_dataset, channel):
    model.eval()
    with torch.no_grad():
        y_true, y_pred = [], []
        for image, label in tqdm(valid_dataset):
            X = image.float().to(device)
            y = label.item()
            _, pred = torch.max(model(X), 1)
            pred = pred.item()
            y_true.append(y)
            y_pred.append(pred)
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
    model.train()
    return f1, accuracy

In [107]:
def train_model(train, test, channel, model, criterion, optimizer, print_every=1):
    print(f"============ Training Starts! ============")
    best_accuracy = 0
    for epoch in range(EPOCHS):
        loss_sum = 0
        for images, label in tqdm(train):
            X = images.float().to(device)
            y = label.to(device)
            
            y_pred = model(X)
            loss = criterion(y_pred, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_sum += loss
            
        if ((epoch % print_every) == 0) or (epoch == (EPOCHS - 1)):
            loss_avg = loss_sum / len(train)
            f1, accuracy = test_eval(model, test, channel)
            print(f">> epoch:[{epoch + 1}/{EPOCHS}] cost: {loss_avg:5.3f} test_accuracy: {accuracy:5.3f} test_f1_score: {f1:5.3f}")
            
    print(f"============ Training Done! ============")

### 3.0. No Augmentation

In [108]:
model = torchvision.models.resnet18(pretrained=False)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 18)
model = model.cuda()

In [109]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss().to(device)

In [110]:
train_model(train_loader, valid_loader, 3, model, criterion, optimizer)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.289 test_accuracy: 0.676 test_f1_score: 0.475



>> epoch:[2/3] cost: 0.630 test_accuracy: 0.758 test_f1_score: 0.573



>> epoch:[3/3] cost: 0.453 test_accuracy: 0.734 test_f1_score: 0.516
============ Training Done! ============


### 3.1. Augmentation 1

In [111]:
model1 = torchvision.models.resnet18(pretrained=False)
n_features = model1.fc.in_features
model1.fc = nn.Linear(n_features, 18)
model1 = model1.cuda()

In [112]:
optimizer1 = optim.Adam(model1.parameters(), lr=LEARNING_RATE)
criterion1 = nn.CrossEntropyLoss().to(device)

In [113]:
train_model(train_loader1, valid_loader1, 3, model1, criterion1, optimizer1)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.092 test_accuracy: 0.745 test_f1_score: 0.550



>> epoch:[2/3] cost: 0.591 test_accuracy: 0.788 test_f1_score: 0.600



>> epoch:[3/3] cost: 0.438 test_accuracy: 0.788 test_f1_score: 0.681
============ Training Done! ============


### 3.2. Augmentation 2

In [114]:
model2 = torchvision.models.resnet18(pretrained=False)
n_features = model2.fc.in_features
model2.fc = nn.Linear(n_features, 18)
model2 = model2.cuda()

In [115]:
optimizer2 = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
criterion2 = nn.CrossEntropyLoss().to(device)

In [116]:
train_model(train_loader2, valid_loader2, 3, model2, criterion2, optimizer2)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.048 test_accuracy: 0.755 test_f1_score: 0.546



>> epoch:[2/3] cost: 0.551 test_accuracy: 0.783 test_f1_score: 0.653



>> epoch:[3/3] cost: 0.370 test_accuracy: 0.842 test_f1_score: 0.763
============ Training Done! ============


### 3.3. Augmentation 3

In [120]:
model3 = torchvision.models.resnet18(pretrained=False)
n_features = model3.fc.in_features
model3.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3.fc = nn.Linear(n_features, 18)
model3 = model3.cuda()

In [121]:
optimizer3 = optim.Adam(model3.parameters(), lr=LEARNING_RATE)
criterion3 = nn.CrossEntropyLoss().to(device)

In [122]:
train_model(train_loader3, valid_loader3, 1, model3, criterion3, optimizer3)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.135 test_accuracy: 0.623 test_f1_score: 0.415



>> epoch:[2/3] cost: 0.543 test_accuracy: 0.791 test_f1_score: 0.676



>> epoch:[3/3] cost: 0.338 test_accuracy: 0.862 test_f1_score: 0.772
============ Training Done! ============


### 3.4. Augmentation 4

In [123]:
model4 = torchvision.models.resnet18(pretrained=False)
n_features = model4.fc.in_features
model4.fc = nn.Linear(n_features, 18)
model4 = model4.cuda()

In [124]:
optimizer4 = optim.Adam(model4.parameters(), lr=LEARNING_RATE)
criterion4 = nn.CrossEntropyLoss().to(device)

In [125]:
train_model(train_loader4, valid_loader4, 3, model4, criterion4, optimizer4)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.113 test_accuracy: 0.742 test_f1_score: 0.575



>> epoch:[2/3] cost: 0.547 test_accuracy: 0.752 test_f1_score: 0.605



>> epoch:[3/3] cost: 0.363 test_accuracy: 0.826 test_f1_score: 0.701
============ Training Done! ============


### 3.5. Augmentation 5

In [126]:
model5 = torchvision.models.resnet18(pretrained=False)
n_features = model5.fc.in_features
model5.fc = nn.Linear(n_features, 18)
model5 = model5.cuda()

In [127]:
optimizer5 = optim.Adam(model5.parameters(), lr=LEARNING_RATE)
criterion5 = nn.CrossEntropyLoss().to(device)

In [128]:
train_model(train_loader5, valid_loader5, 3, model5, criterion5, optimizer5)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.068 test_accuracy: 0.731 test_f1_score: 0.527



>> epoch:[2/3] cost: 0.536 test_accuracy: 0.648 test_f1_score: 0.567



>> epoch:[3/3] cost: 0.370 test_accuracy: 0.762 test_f1_score: 0.645
============ Training Done! ============


### 3.6. Augmentation 6

In [129]:
model6 = torchvision.models.resnet18(pretrained=False)
n_features = model6.fc.in_features
model6.fc = nn.Linear(n_features, 18)
model6 = model6.cuda()

In [130]:
optimizer6 = optim.Adam(model6.parameters(), lr=LEARNING_RATE)
criterion6 = nn.CrossEntropyLoss().to(device)

In [131]:
train_model(train_loader6, valid_loader6, 3, model6, criterion6, optimizer6)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.099 test_accuracy: 0.647 test_f1_score: 0.459



>> epoch:[2/3] cost: 0.548 test_accuracy: 0.537 test_f1_score: 0.481



>> epoch:[3/3] cost: 0.378 test_accuracy: 0.857 test_f1_score: 0.744
============ Training Done! ============


### 3.7. Augmentation 7

In [132]:
model7 = torchvision.models.resnet18(pretrained=False)
n_features = model7.fc.in_features
model7.fc = nn.Linear(n_features, 18)
model7 = model7.cuda()

In [133]:
optimizer7 = optim.Adam(model7.parameters(), lr=LEARNING_RATE)
criterion7 = nn.CrossEntropyLoss().to(device)

In [134]:
train_model(train_loader7, valid_loader7, 3, model7, criterion7, optimizer7)

============ Training Starts! ============



>> epoch:[1/3] cost: 1.105 test_accuracy: 0.748 test_f1_score: 0.540



>> epoch:[2/3] cost: 0.551 test_accuracy: 0.804 test_f1_score: 0.697



>> epoch:[3/3] cost: 0.369 test_accuracy: 0.823 test_f1_score: 0.687
============ Training Done! ============
